In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import IPython as Ip
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from sklearn.utils import shuffle
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import pickle
import os

In [2]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join("D:", os.path.sep, "Web Development", "Leaf"),
    shuffle=True,
    batch_size=32,
    image_size=(224,224),
)
labels = np.array(dataset.class_names).reshape(-1,1)

Found 111265 files belonging to 449 classes.


In [8]:
def get_dataset(ds,train_size=0.8,test_size=0.2,shuffles=10000):
    if shuffles:
        ds.shuffle(shuffles,seed=42)
    train_size = int(train_size*len(ds))
    test_size = int(test_size * len(ds))
    train_ds = ds.take(train_size)
    test_ds = ds.skip(train_size)
    val_ds = test_ds.take(int(test_size*0.4))
    return train_ds,test_ds,val_ds

train_dataset,test_dataset,val_dataset = get_dataset(dataset)
print(len(train_dataset),len(test_dataset),len(val_dataset))

174 44 17


In [7]:
DenseNet_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='max')

for layer in DenseNet_model.layers:
    layer.trainable = True

x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(DenseNet_model.output)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dense(1024, activation='relu')(x)

output = Dense(len(labels), activation='softmax')(x)

model = Model(inputs=DenseNet_model.input, outputs=output)

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
history = model.fit(train_dataset,
                validation_data=val_dataset,
                #callbacks=[early_stop],
                epochs=30,
                batch_size=32            
                )

Epoch 1/30


In [3]:
#with open('Cnn.pickle','wb') as saved_model:
#    pickle.dump(model,saved_model)
with open('Cnn.pickle','rb') as saved_model:
    loaded_model = pickle.load(saved_model)

In [8]:
test = loaded_model.evaluate(test_dataset)

696/696 ━━━━━━━━━━━━━━━━━━━━ 1679s 2s/step - accuracy: 0.6094 - loss: 1.5207


In [9]:
test

[1.5313717126846313, 0.6068522334098816]

In [4]:
def prediction_with_loaded_model(imgs):
    imgs = tf.keras.preprocessing.image.load_img(imgs,target_size=(224,224))
    imgs = tf.keras.preprocessing.image.img_to_array(imgs)
    imgs = tf.expand_dims(imgs,0)
    
    predict = loaded_model.predict(imgs)
    score = tf.nn.sigmoid(predict[0])
    
    print(f'This image likely belongs to {labels[np.argmax(score)]} with the accuracy of {100 * np.max(score)}% .')

In [11]:
prediction_with_loaded_model(r"D:\Images\bamboo2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
This image likely belongs to ['CallaLily'] with the accuracy of 61.59018278121948% .


In [12]:
prediction_with_loaded_model(r"D:\Images\mango.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
This image likely belongs to ['Amruthaballi'] with the accuracy of 60.99408268928528% .


In [13]:
prediction_with_loaded_model(r"D:\Images\mango.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
This image likely belongs to ['Amruthaballi'] with the accuracy of 60.99408268928528% .


In [14]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\Hibiscus_flower_TZ.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
This image likely belongs to ['Poinsettia'] with the accuracy of 54.446154832839966% .


In [15]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\poinsettia.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
This image likely belongs to ['Poinsettia'] with the accuracy of 73.07265400886536% .


In [16]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\images.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
This image likely belongs to ['Amaryllis'] with the accuracy of 70.87454199790955% .


In [17]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\Amarylis.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
This image likely belongs to ['Amaryllis'] with the accuracy of 72.77393937110901% .


In [18]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\Tulsi.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
This image likely belongs to ['Nithyapushpa'] with the accuracy of 62.69795894622803% .


In [19]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\Nithyapushpa.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
This image likely belongs to ['MoonflowerVine'] with the accuracy of 61.12087368965149% .


In [20]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\MoonflowerVine.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
This image likely belongs to ['MoonflowerVine'] with the accuracy of 72.00932502746582% .


In [21]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\Hybiscus2.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
This image likely belongs to ['Amaryllis'] with the accuracy of 68.5032844543457% .


In [22]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\Hyb.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
This image likely belongs to ['Poinsettia'] with the accuracy of 67.9844856262207% .


In [23]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\rose.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
This image likely belongs to ['Roses'] with the accuracy of 60.69513559341431% .


In [24]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\lotus.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
This image likely belongs to ['OrientalLily'] with the accuracy of 61.335289478302% .


In [25]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\orientallily.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
This image likely belongs to ['OrientalLily'] with the accuracy of 57.92946815490723% .


In [26]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\sunflower.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
This image likely belongs to ['Calendula'] with the accuracy of 59.0083122253418% .


In [27]:
prediction_with_loaded_model(r'D:\Web Development\Css-html\calendula.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
This image likely belongs to ['Calendula'] with the accuracy of 72.69576787948608% .


In [94]:
prediction_with_loaded_model(r"D:\Web Development\Leaf\Ganike\20200402_085547_1.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
This image likely belongs to ['Ganike'] with the accuracy of 57.66265392303467% .


In [5]:
prediction_with_loaded_model(r"D:\Web Development\Leaf\BabysBreath\21dd358f72.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
This image likely belongs to ['BabysBreath'] with the accuracy of 71.83054685592651% .


In [6]:
prediction_with_loaded_model(r"D:\Web Development\Leaf\Jasmine\1b55e441fe.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
This image likely belongs to ['Jasmine'] with the accuracy of 61.50609254837036% .
